# Extracción Automática

Este notebook responde la primera pregunta de la prueba técnica:

**"Extraer automáticamente ubicación, fecha y tipo de incidente de cada reporte".**

Pasos:
1. Subir archivos `.zip` de reportes o PDFs individuales.  
2. Extraer PDFs y convertirlos a texto.  
3. Detectar **Ubicaciones** (NER con spaCy).  
4. Detectar **Fechas** (regex en español).  
5. Detectar **Tipo de incidente** (keywords).  
6. Generar una **tabla consolidada** (DataFrame + CSV).


#0 Instalar dependencias

In [1]:
!pip install spacy PyPDF2 pandas
!python -m spacy download es_core_news_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 62.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## 1. Subir y leer reportes


In [2]:
import os
import zipfile
from PyPDF2 import PdfReader

from google.colab import files
uploaded = files.upload()  # Subir ZIP con PDFs o PDFs individuales

# Si subiste un ZIP, descomprime en "data/"
for fname in uploaded.keys():
    if fname.endswith(".zip"):
        with zipfile.ZipFile(fname, 'r') as zip_ref:
            zip_ref.extractall("data")
    else:
        os.makedirs("data", exist_ok=True)
        os.rename(fname, os.path.join("data", fname))

print("Archivos en carpeta data/:", os.listdir("data"))

# Función para leer PDFs en subcarpetas
def read_reports(path="data"):
    reports = {}
    for root, dirs, files in os.walk(path):  # 👈 recorre subcarpetas
        for fname in files:
            if fname.endswith(".pdf"):
                fpath = os.path.join(root, fname)
                pdf = PdfReader(fpath)
                text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
                reports[fname] = text
    return reports

reports = read_reports("data")
print("Archivos cargados:", list(reports.keys()))


Saving Reportes incidentes humanitarios.zip to Reportes incidentes humanitarios.zip
Archivos en carpeta data/: ['Reportes incidentes humanitarios']
Archivos cargados: ['Acceso Hum. Cauca_Guaviare. 12.08.pdf', 'SitRep Emergencia por Inundaciones en Regiones Amazonia Orinoquia.pdf', '_BriefingHumanitario_La Guajira_Enero-Junio2025.pdf']


## 2. Procesar reportes (NER, regex y keywords)


In [7]:
#Definir extraccion
import re
import spacy
import pandas as pd


# Inicializar spaCy
nlp = spacy.load("es_core_news_sm")

def extraer_info_pregunta1(text, nombre_reporte="reporte.pdf"):
    doc = nlp(text)

    # --- Ubicaciones ---
    ubicaciones = [ent.text.strip() for ent in doc.ents if ent.label_ in ["LOC", "GPE"]]
    ubicaciones_limpias = []
    for loc in ubicaciones:
        loc = re.sub(r"\n", " ", loc)
        loc = re.sub(r"([a-z])([A-Z])", r"\1, \2", loc)
        ubicaciones_limpias.extend([l.strip() for l in loc.split(",") if len(l.strip()) > 2])
    ubicaciones_final = list(set(ubicaciones_limpias))

    # --- Fechas ---
    regex_patterns = [
        # Ej: "10 de mayo 2023"
        r"\b\d{1,2}\s+de\s+(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+\d{4}\b",

        # Ej: "mayo 2023"
        r"\b(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+\d{4}\b",

        # Ej: "abril-junio 2023"
        r"\b(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s*[-–]\s*(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+\d{4}\b",

        # Ej: "10/01/2023" o "20-01-2023"
        r"\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b"
    ]
    fechas_regex = []
    for pattern in regex_patterns:
        matches = re.findall(pattern, text, flags=re.IGNORECASE)
        fechas_regex.extend(matches)
    fechas_final = list(set(fechas_regex))

    # --- Tipos de incidente ---
    keywords_incidentes = {
        "Inundación": ["inundación", "lluvia", "río", "desbordamiento"],
        "Desplazamiento": ["desplazamiento", "huida", "migración"],
        "Salud": ["epidemia", "enfermedad", "hospital", "salud"],
        "Seguridad": ["conflicto", "violencia", "ataque", "amenaza"]
    }
    tipos_detectados = []
    for tipo, palabras in keywords_incidentes.items():
        if any(palabra.lower() in text.lower() for palabra in palabras):
            tipos_detectados.append(tipo)

    return {
        "Reporte": nombre_reporte,
        "Ubicaciones": ", ".join(ubicaciones_final[:15]) if ubicaciones_final else "No detectadas",
        "Fechas": ", ".join(fechas_final) if fechas_final else "No detectadas",
        "Tipos de incidente": ", ".join(tipos_detectados) if tipos_detectados else "No detectados"
    }


## 3. Generar tabla consolidada


In [8]:
# Procesar todos los reportes
resultados = []
for fname, contenido in reports.items():
    resultados.append(extraer_info_pregunta1(contenido, nombre_reporte=fname))

# DataFrame
df_resultados = pd.DataFrame(resultados)



# Guardar CSV
os.makedirs("outputs", exist_ok=True)
df_resultados.to_csv("outputs/tabla_pregunta1.csv", index=False, encoding="utf-8")
print("✔ Tabla exportada en outputs/tabla_pregunta1.csv")

# Mostrar tabla en Colab
df_resultados

✔ Tabla exportada en outputs/tabla_pregunta1.csv


,Reporte,Ubicaciones,Fechas,Tipos de incidente
0,Acceso Hum. Cauca_Guaviare. 12.08.pdf,"Corinto, Miraflores, Argelia, La Guajira Sucre...","11 de agosto 2025, agosto 2025","Inundación, Desplazamiento, Seguridad"
1,SitRep Emergencia por Inundaciones en Regiones...,"Colombia, Además, Cubarral, Miraflores, R4V.in...","21/02/2020, 29/07/2025","Inundación, Desplazamiento, Salud, Seguridad"
2,_BriefingHumanitario_La Guajira_Enero-Junio202...,"Histórico, Además, Colombia, Comité Departamen...","junio 2025, junio 2025","Inundación, Desplazamiento, Salud, Seguridad"
